In [19]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [20]:
# Enable autoreloading if import packages are changed
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import glob
import os
from src.match import match

import os
from src.utils import utils

PROJECT_DIR = os.getcwd()

In [22]:
# Editable part
dataset_name = "milk_stage2"

In [23]:
# info = utils.import_json(f"./dataset/{dataset_name}/info.json")
dataset_info = utils.get_dataset_info(dataset_name, PROJECT_DIR)
dataset_info["data"]

{'path': '/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2',
 'image': '/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/images',
 'correspondence': '/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/correspondences'}

### Generate new correspondences

In [24]:
class CorrespondencesGenerator:
    def __init__(self, dataset_info):
        self.dataset_info = dataset_info
        img_idx = np.array([int(os.path.basename(i).split(".")[0]) for i in glob.glob(os.path.join(dataset_info["data"]["image"], "*.jpg"))])
        img_idx = sorted(img_idx)
        self.images_paths = dict([(idx, os.path.join(dataset_info["data"]["image"], "{:05}.jpg".format(idx))) for idx in img_idx])
        self.correspondences_exists = glob.glob(os.path.join(dataset_info["data"]["correspondence"], "*.txt"))

    def save_correspondences(self, idx_l, idx_r, data_list):
        save_path = os.path.join(self.dataset_info["data"]["correspondence"], f"{idx_l}_{idx_r}.txt")
        np.savetxt(save_path, data_list)
corr = CorrespondencesGenerator(dataset_info)

In [25]:
# utilizes SIFT as detector
detector = cv2.SIFT_create()
max_img_ahead = 35

for i, (idx1, img_path1) in enumerate(corr.images_paths.items()):
    for idx2, img_path2 in zip(list(corr.images_paths.keys())[i:i+max_img_ahead], list(corr.images_paths.values())[i:i+max_img_ahead]):
        if idx1 == idx2:
            continue

        # extract gray color
        img1 = cv2.imread(img_path1)
        color1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        gray1 = cv2.cvtColor(color1, cv2.COLOR_RGB2GRAY)
        img2 = cv2.imread(img_path2)
        color2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        gray2 = cv2.cvtColor(color2, cv2.COLOR_RGB2GRAY)

        # extract kp and desc
        keypoints1, desc1 = detector.detectAndCompute(gray1, None)
        keypoints2, desc2 = detector.detectAndCompute(gray2, None)
        result = match(torch.tensor(desc1), torch.tensor(desc2), "cpu", threshold=200, ratio=0.75)

        # store correspondences of images
        corr_imgs = np.array([[keypoints1[i].pt[0], keypoints1[i].pt[1], keypoints2[j].pt[0], keypoints2[j].pt[1]] for [i, j] in result])
        corr.save_correspondences(idx1, idx2, corr_imgs)
        print(idx1, idx2)

0 28
0 56
0 84
0 113
0 141
0 169
0 198
0 226
0 254
0 283
0 311
0 339
0 368
0 396
0 424
0 453
0 481
0 509
0 538
0 566
0 585
0 607
0 651
0 679
0 708
0 736
0 764
0 792
0 821
0 849
0 877
0 906
0 934
0 962
28 56
28 84
28 113
28 141
28 169
28 198
28 226
28 254
28 283
28 311
28 339
28 368
28 396
28 424
28 453
28 481
28 509
28 538
28 566
28 585
28 607
28 651
28 679
28 708
28 736
28 764
28 792
28 821
28 849
28 877
28 906
28 934
28 962
28 991
56 84
56 113
56 141
56 169
56 198
56 226
56 254
56 283
56 311
56 339
56 368
56 396
56 424
56 453
56 481
56 509
56 538
56 566
56 585
56 607
56 651
56 679
56 708
56 736
56 764
56 792
56 821
56 849
56 877
56 906
56 934
56 962
56 991
56 1019
84 113
84 141
84 169
84 198
84 226
84 254
84 283
84 311
84 339
84 368
84 396
84 424
84 453
84 481
84 509
84 538
84 566
84 585
84 607
84 651
84 679
84 708
84 736
84 764
84 792
84 821
84 849
84 877
84 906
84 934
84 962
84 991
84 1019
84 1047
113 141
113 169
113 198
113 226
113 254
113 283
113 311
113 339
113 368
113 396
113 4